In [37]:
# Import the Natural Language Toolkit library for text processing
import nltk
# Import WordNetLemmatizer for reducing words to their base form
from nltk.stem import WordNetLemmatizer
# Initialize the lemmatizer object
lemmatizer = WordNetLemmatizer()
# Import pickle for loading serialized Python objects
import pickle
# Import numpy for numerical operations
import numpy as np

# Import the Keras function to load a pre-trained model
from tensorflow.keras.models import load_model
# Load the chatbot model from a saved file
model = load_model('chatbot_model.h5')
# Import json for parsing JSON data
import json
# Import random for generating random selections
import random
# Load the questions/intents data from a JSON file
quests = json.loads(open('quest.json').read())
# Load the vocabulary (bag of words) from a pickle file
words = pickle.load(open('words1.pkl','rb'))
# Load the intent categories/classes from a pickle file
classes = pickle.load(open('classes1.pkl','rb'))

In [39]:
def clean_up_sentence(sentence):
    # Tokenize the pattern - split sentence into individual words
    sentence_words = nltk.word_tokenize(sentence)
    # Lemmatize each word - reduce words to their base/root form and convert to lowercase
    sentence_words = [lemmatizer.lemmatize(word.lower()) for word in sentence_words]
    return sentence_words

# Convert a sentence into a bag of words representation (binary vector)
def bow(sentence, words, show_details=True):
    # Process the input sentence into tokens and lemmatized words
    sentence_words = clean_up_sentence(sentence)
    # Initialize the bag vector with zeros (length = vocabulary size)
    bag = [0]*len(words)  
    for s in sentence_words:
        for i,w in enumerate(words):
            if w == s: 
                # Mark the word as present (1) in the bag vector at the vocabulary position
                bag[i] = 1
                if show_details:
                    print ("found in bag: %s" % w)
    # Return the bag of words as a numpy array
    return(np.array(bag))

In [41]:
def predict_class(sentence, model):
    # Convert input sentence to bag-of-words representation
    p = bow(sentence, words, show_details=False)
    
    # Make prediction using the neural network model
    res = model.predict(np.array([p]))[0]
    
    # Set minimum confidence threshold for predictions
    ERROR_THRESHOLD = 0.25
    
    # Filter predictions that exceed the confidence threshold
    results = [[i,r] for i,r in enumerate(res) if r>ERROR_THRESHOLD]
    
    # Sort predictions by probability in descending order
    results.sort(key=lambda x: x[1], reverse=True)
    
    # Format results as a list of dictionaries with intent class and probability
    return_list = []
    for r in results:
        return_list.append({"quest": classes[r[0]], "probability": str(r[1])})
    return return_list

def getResponse(ints, quests_json):
    # Extract the highest confidence intent tag
    tag = ints[0]['quest']
    
    # Get the list of all intents from the JSON data
    list_of_quests = quests_json['quests']
    
    # Find the matching intent and select a random response
    for i in list_of_quests:
        if(i['tag']== tag):
            result = random.choice(i['responses'])
            break
    return result

def chatbot_response(msg):
    # Process user input to identify intent
    ints = predict_class(msg, model)
    
    # Get appropriate response based on identified intent
    res = getResponse(ints, quests)
    return res

In [43]:
# Creating GUI with tkinter
import tkinter as tk
from tkinter import *


def send():
    # Get message from the entry box and remove whitespace
    msg = EntryBox.get("1.0",'end-1c').strip()
    # Clear the entry box
    EntryBox.delete("0.0",END)

    if msg != '':
        # Enable the chat log for editing
        ChatLog.config(state=NORMAL)
        # Add user message to chat log
        ChatLog.insert(END, "You: " + msg + '\n\n')
        # Set text formatting for the chat log
        ChatLog.config(foreground="#442265", font=("Verdana", 12 ))
    
        # Get response from chatbot
        res = chatbot_response(msg)
        # Add bot response to chat log
        ChatLog.insert(END, "Bot: " + res + '\n\n')
            
        # Disable chat log for editing
        ChatLog.config(state=DISABLED)
        # Auto-scroll to the end of chat log
        ChatLog.yview(END)
 

# Create the main window
base = Tk()
# Set window title
base.title("Chatbot")
# Set window dimensions
base.geometry("600x550")
# Disable window resizing
base.resizable(width=FALSE, height=FALSE)

''

In [29]:
#Create Chat window

ChatLog = Text(base, bd=0, bg="white", height="240", width="150", font="Arial")

ChatLog.config(state=DISABLED)
#label = tk.Label(root, text="Hi! How may I help you?")
#Bind scrollbar to Chat window
scrollbar = Scrollbar(base, command=ChatLog.yview, cursor="heart")
ChatLog['yscrollcommand'] = scrollbar.set

#Create Button to send message
SendButton = Button(base, font=("Verdana",12,'bold'), text="Send", width="12", height=5,
                    bd=0, bg="#25cdf7", activebackground="#3c9d9b",fg='#ffffff',
                    command= send )

#Create the box to enter message
EntryBox = Text(base, bd=0, bg="white",width="99", height="5", font="Arial")
tk.Label(text="Hi!").grid(column=0, row=0)


#Place all components on the screen
scrollbar.place(x=575,y=6, height=450)
ChatLog.place(x=6,y=6, height=450, width=570)
EntryBox.place(x=6, y=470, height=100, width=460)
SendButton.place(x=455, y=470, height=100)
base.mainloop()

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
